## 1. Import

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings(action='ignore')

## 2. Data Load

In [15]:
# 학습/평가 데이터 로드
train_df = pd.read_csv('C:/Project/DACON_DEFAULT_PREDICTION/data/default_prediction/train.csv').drop(columns=['UID'])
test_df = pd.read_csv('C:/Project/DACON_DEFAULT_PREDICTION/data/default_prediction/test.csv').drop(columns=['UID'])

### 2-1. 데이터 조회

In [16]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   주거 형태              10000 non-null  object 
 1   연간 소득              10000 non-null  float64
 2   현재 직장 근속 연수        10000 non-null  object 
 3   체납 세금 압류 횟수        10000 non-null  float64
 4   개설된 신용계좌 수         10000 non-null  int64  
 5   신용 거래 연수           10000 non-null  float64
 6   최대 신용한도            10000 non-null  float64
 7   신용 문제 발생 횟수        10000 non-null  int64  
 8   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 9   개인 파산 횟수           10000 non-null  int64  
 10  대출 목적              10000 non-null  object 
 11  대출 상환 기간           10000 non-null  object 
 12  현재 대출 잔액           10000 non-null  float64
 13  현재 미상환 신용액         10000 non-null  float64
 14  월 상환 부채액           10000 non-null  float64
 15  신용 점수              10000 non-null  int64  
 16  채무 불이행 여부          1000

In [17]:
train_df.describe()

,연간 소득,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
count,1.000000e+04,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.00000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.00000,10000.000000,10000.000000
mean,2.163959e+06,0.199300,12.248900,19.879360,1.175265e+06,0.62620,30.889200,0.373200,5.061200e+05,3.649126e+05,22367.28075,744.215000,0.341200
std,1.434430e+06,0.714304,4.620572,7.206693,1.604199e+06,1.23419,20.011561,0.843797,2.831462e+05,3.537942e+05,15186.49738,56.995698,0.474136
min,2.676210e+05,0.000000,2.000000,6.000000,0.000000e+00,0.00000,0.000000,0.000000,2.917650e+04,0.000000e+00,0.00000,502.000000,0.000000
25%,1.311437e+06,0.000000,9.000000,14.600000,4.482476e+05,0.00000,14.000000,0.000000,3.001901e+05,1.471856e+05,10893.75000,704.000000,0.000000
50%,1.743223e+06,0.000000,12.000000,17.950000,7.670910e+05,0.00000,28.000000,0.000000,4.743412e+05,2.547930e+05,20160.00000,756.000000,0.000000
75%,2.447664e+06,0.000000,15.000000,24.100000,1.147283e+06,1.00000,41.000000,0.000000,5.919079e+05,4.749180e+05,30647.25000,793.000000,1.000000
max,1.722975e+07,7.000000,35.000000,51.700000,2.323233e+07,7.00000,88.000000,3.000000,1.373613e+06,3.946300e+06,153574.50000,825.000000,1.000000


In [18]:
h_t = pd.concat([train_df.head(), train_df.tail()]).reset_index(drop=True)
h_t

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0
5,주택 담보 대출 (거주 중),1339473.0,10년 이상,0.0,9,18.7,319027.5,0,68,0,부채 통합,단기 상환,126216.0,177028.5,6237.0,755,0
6,주택 담보 대출 (거주 중),2297230.5,2년,0.0,11,28.3,399799.5,0,7,0,주택 개보수,장기 상환,371907.0,347449.5,53301.0,707,0
7,주택 담보 대출 (거주 중),1221523.5,10년 이상,0.0,9,30.1,823305.0,0,14,0,부채 통합,장기 상환,869736.0,176905.5,11436.0,733,0
8,자가,3343584.0,10년 이상,0.0,10,20.3,724314.0,0,25,0,부채 통합,단기 상환,443008.5,139294.5,25567.5,696,0
9,주택 담보 대출 (거주 중),2175133.5,5년,0.0,5,24.9,52999.5,0,9,0,주택 개보수,장기 상환,1152918.0,106930.5,12676.5,676,0


## 3. Pre-processing (전처리)

In [19]:
categorical_col = [
    '주거 형태',
    '현재 직장 근속 연수',
    '대출 목적',
    '대출 상환 기간'
]

# OneHotEncoder 초기화
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 훈련 데이터에 대해 인코더 학습
encoder.fit(train_df[categorical_col])

# 훈련 데이터와 테스트 데이터 변환
train_encoded = encoder.transform(train_df[categorical_col])
test_encoded = encoder.transform(test_df[categorical_col])

# One-hot encoding 결과를 데이터프레임으로 변환
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(categorical_col))
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(categorical_col))

# 인코딩된 결과를 원래 데이터에 적용
train_df = pd.concat([train_df.drop(columns=categorical_col).reset_index(drop=True), train_encoded_df], axis=1)
test_df = pd.concat([test_df.drop(columns=categorical_col).reset_index(drop=True), test_encoded_df], axis=1)

In [20]:
abs(train_df.corr(numeric_only=True)['채무 불이행 여부']).drop('채무 불이행 여부').sort_values()

현재 미상환 신용액                0.000048
대출 목적_휴가 비용               0.000407
마지막 연체 이후 경과 개월 수         0.012058
대출 목적_교육비                 0.012467
현재 직장 근속 연수_7년            0.013827
대출 목적_이사 비용               0.014396
주거 형태_자가                  0.015493
대출 목적_결혼 자금               0.015664
현재 직장 근속 연수_1년 미만         0.019085
현재 직장 근속 연수_6년            0.019543
월 상환 부채액                  0.020135
대출 목적_주택 구매               0.020176
현재 직장 근속 연수_3년            0.027998
현재 직장 근속 연수_9년            0.029462
체납 세금 압류 횟수               0.030419
현재 대출 잔액                  0.033052
주거 형태_주택 담보 대출 (비거주 중)    0.033834
대출 목적_소규모 사업 자금           0.036743
대출 목적_의료비                 0.038749
대출 목적_고액 구매               0.039015
현재 직장 근속 연수_5년            0.039539
대출 목적_자동차 구매              0.042213
신용 문제 발생 횟수               0.044847
최대 신용한도                   0.044997
대출 목적_여행 자금               0.050192
현재 직장 근속 연수_4년            0.050732
대출 목적_사업 대출               0.053173
주거 형태_주택 담보 대출 (거주 중)     0.058143
현재 직장 근속 연수_2년      

## 4. Train / Validation Split (학습 데이터 분할)

In [21]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df.drop(columns=['채무 불이행 여부']), 
    train_df['채무 불이행 여부'], 
    test_size=0.2, 
    random_state=42
)

## 5. Model Training (모델 학습)

In [22]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import *

In [23]:
model_param_dict = {
    'LogisticRegression': {
        'model' : LogisticRegression(),
        'params' : {
            "C" : [0.01, 0.1, 1, 10, 100]
        }
    },
    "XGBClassifier" : {
        'model' : XGBClassifier(),
        'params' : {
            'n_estimators': [100, 300, 500],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.05, 0.1]
        }
    },
    'LGBMClassifier' : {
        'model' : LGBMClassifier(),
        'params' : {
            'n_estimators': [100, 300, 500],
            'learning_rate': [0.01, 0.05, 0.1]
        }
    },
    "RandomForestClassifier" : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators' : [100, 300, 500],
            'max_depth' : [None, 10, 20, 30],
            'min_samples_leaf' : [2, 5, 10]
        }
    }
}
model_param_dict.keys()

dict_keys(['LogisticRegression', 'XGBClassifier', 'LGBMClassifier', 'RandomForestClassifier'])

## 6. Prediction

In [24]:
def evaluate(model,param,X_train,y_train, X_test, y_test):
    grid_search = GridSearchCV(
        model,
        param,
        cv=5,
        scoring='roc_auc',
        n_jobs=-1
    )

    grid_search.fit(X_train,y_train)
    # best model 
    best_model = grid_search.best_estimator_

    # predict
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:,1]

    auc = roc_auc_score(y_test,y_proba)

    return best_model, auc

In [25]:
model_dict = {}
score = 0

for model in model_param_dict.keys():
    model_dict[model] = evaluate(model_param_dict[model]['model'],
                                 model_param_dict[model]['params'],
                                 X_train,
                                 y_train,
                                 X_val,
                                 y_val
                                 )
    
    print(f"{model}_auc : {model_dict[model][1]}")
    if score < model_dict[model][1]:
        mymodel= model_dict[model][0]
        score = model_dict[model][1]

LogisticRegression_auc : 0.5999126307841657
XGBClassifier_auc : 0.7424668320569517
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2682, number of negative: 5318
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001160 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1879
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335250 -> initscore=-0.684535
[LightGBM] [Info] Start training from score -0.684535
LGBMClassifier_auc : 0.7352766691834755
RandomForestClassifier_auc : 0.739939596591522


In [29]:
# 채무 불이행 '확률'을 예측합니다.
preds = mymodel.predict_proba(test_df)[:,1]
# print(roc_auc_score(test_df,preds))

## 7. Submission (제출 파일 생성)

In [28]:
submit = pd.read_csv('C:/Project/DACON_DEFAULT_PREDICTION/data/default_prediction/sample_submission.csv')

# 결과 저장
submit['채무 불이행 확률'] = preds
submit.to_csv('./submission.csv', encoding='UTF-8-sig', index=False)